<a href="https://colab.research.google.com/github/ml-projects/knowledge-base/blob/master/Group6_Venky_M0_MP1_Data_Munging_(Ungraded).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science

##  A program by IISc and TalentSprint

### Mini Project Notebook 1 : Data munging

(Ungraded Mini-Project)

## Learning Objectives



At the end of the experiment, you will be able to


* understand the requirements for a “clean” dataset, ready for use in statistical analysis.

* use Python libraries like Pandas, Numpy, and Matplotlib to perform the  data-preprocessing steps accordingly.

* derive meaningful insights from the data


## Dataset

The dataset chosen for this experiment is **play store** dataset which is  publicly available and created with this [methodology](https://nycdatascience.com/blog/student-works/google-play-store-everything-that-you-need-to-know-about-the-android-market/)  

This dataset consists of 10841 records. Each record is made up of 13 fields.

**For example**, one record consist of App, Category, Rating, Reviews, Size, Installs, Type, Price, Content Rating, Genres, Last Updated, Current Ver, and Android Ver.

## Problem Statement

Before we can derive any meaningful insights from the Play Store data, it is essential to pre-process the data and make it suitable for further analysis. This pre-processing step forms a major part of data wrangling (or data munging) and ensures better quality data. It consists of the transformation and mapping of data from a "raw" data form into another format so that it is more valuable for a variety of downstream purposes such as analytics. Data analysts typically spend a sizeable amount of time in the process of data wrangling, compared to the actual analysis of the data.

After data munging is performed, several actionable insights can be derived from the Play Store apps data. Such insights could help to unlock the enormous potential to drive app-making businesses to success.

In [ ]:
#@title Download the data
!wget -qq https://cdn.iisc.talentsprint.com/CDS/Datasets/googleplaystore.csv
print("Data downloaded successfully!")

#### Load the dataset

In [ ]:
import pandas as pd

DATAFILE = "googleplaystore.csv"
#load playstore data into a pandas data frame
data = pd.read_csv(DATAFILE)

## Pre-processing

There are different steps involved in Data Preprocessing. These steps are as follows:

    1. Data Cleaning → In this step the primary focus is on
        -Handling missing data
        -Handling noisy data
        -Detection and removal of outliers
    
    2. Data Integration → This process is used when data is gathered from various data sources
    and data are combined to form consistent data. This data after performing cleaning is used
    for analysis.
    
    3. Data Transformation → In this step we will convert the raw data into a specified format
    according to the need of the model we are building. There are many options used for
    transforming the data as below:
        -Normalization
        -Aggregation
        -Generalization
        
    4. Data Reduction → After data transformation and scaling the redundancy within the data
    is removed and efficiently organizing the data is performed.

### Task 1: Data Cleaning

* Check whether there are any null values and figure out how you want to handle them? 
  
    **Hint:** isnan(), dropna(), fillna()
* If there is any duplication of a record, how would you like to handle it?

    Hint: [drop_duplicates](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html)

* Are there any non-English apps? And how to filter them?

* In the size column, multiply 1,000,000 with M in the cell and multiply by 1000 if we have K in the cell.

In [ ]:
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)
data.shape

In [ ]:
import regex as re

def remove_emoji(text:str) -> str:
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text).strip()

def isEnglish(appname: str) -> bool:
  return remove_emoji(appname).isascii()


In [ ]:
data['isAppNameEng'] = data.App.apply(isEnglish)
data = data[(data.isAppNameEng == True)]

In [ ]:
#@title Transforming by vector way
mask = data.Size.str.upper().str.endswith('M')
data.loc[mask,'Size_new'] = data[mask].Size.str[:-1].astype(float) * 100000 
mask = data.Size.str.upper().str.endswith('K')
data.loc[mask,'Size_new'] = data[mask].Size.str[:-1].astype(float) * 1000
#data.fillna(value={'Size_new': 0}, inplace=True)
data.dropna(inplace=True)
data['Size'] = data['Size_new']
data.drop(labels=['Size_new'], axis=1, inplace=True)
data[data.Size.isna()].head(20)
data.shape

In [ ]:
#remove apps with in valid size
#data = data[(data.Size > 0)]

In [ ]:
data.shape

In [ ]:

data.loc[:,"Installs"] = data.Installs.str.replace(',','').str.replace('+','').astype(int)


In [ ]:
data.loc[:,"Price"] = data.Price.str.replace(',','').str.replace('$','').astype(float)

## Visualization

### Task 2: Perform the following:

##### Exercise 1: Find the number of apps in various categories by using an appropriate plot.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [ ]:
# dataframe to hold count by category
catdf = data.groupby("Category").agg(num_of_apps = ('App','nunique'))
sns.set_style("darkgrid")
catdf.plot(kind="bar",figsize=(20,6),title="Number of Applications by Category")
plt.show()

##### Exercise 2: Explore the distribution of free and paid apps across different categories

**Hint:** Stacked Bar Chart

In [ ]:
# Group by Category and Type, use unique count aggregation on column "APP"
TypeDistrDF = data.groupby(by=["Category","Type"]).agg(num_of_apps = ('App','nunique')).reset_index()
TypeDistrDF

In [ ]:
# Transpose data
TypeDistrDF= TypeDistrDF.pivot(index='Category',columns = 'Type',values='num_of_apps').fillna(value =0 )
TypeDistrDF

In [ ]:
sns.set_style("darkgrid")
TypeDistrDF.plot(kind="bar",stacked=True,figsize=(20,5),title="App Distribution by Type and Category",ylabel="Count of Apps") # by default index will be plotted on x-axis
plt.show()

##### Exercise 3: Represent the distribution of app rating on a scale of 1-5 using an appropriate plot

**Hint:** histogram / strip plot

In [ ]:
sns.set_style("white")
data.hist(column='Rating',bins = 5,grid=False,figsize=(20,5),align='mid',histtype='stepfilled',label=None)
plt.title("Distribution of Applications Rating")
plt.ylabel("Number of Apps")
plt.xlabel("Rating")
plt.show()

##### Exercise 4: Identify outliers of the rating column by plotting the boxplot category wise and Handle them.

**Hint:** Removing Outliers using z-score, quantile [link](https://kanoki.org/2020/04/23/how-to-remove-outliers-in-python/) 

In [ ]:
outlierratingsdf = data.loc[:,{"Category","Rating"}]
outlierratingsdf

In [ ]:
outlierratingsdf.boxplot(column='Rating',by='Category',figsize=(10,10),vert= False,patch_artist=True)
plt.title("Box Plot for Outlier Identification by Categories")
plt.ylabel("Category")
plt.xlabel("Rating")
plt.show()

##### Exercise 5: Plot the barplot of all the categories indicating no. of installs

In [ ]:
sns.set_style('darkgrid')
installsDF = data.groupby("Category").agg(num_of_installs = ('Installs','sum'))
installsDF.plot(kind="bar",figsize=(20,10),logy='sym',ylabel='Number of Installs',title='Number of Installations by Category')
plt.show()

## Insights


### Task 3: Derive the below insights

##### Exercise 1: Does the price correlate with the size of the app?

  **Hint:** plot the scatterplot of `Size` and `Price`

In [ ]:

size_price_df = data.loc[:,{"Size","Price"}]
size_price_df.plot(kind="scatter",x="Price",y="Size",figsize=(20,10),logy='sym',title="Correlation between size and price of Apps")
plt.show()



In [ ]:
# Compute the correlation matrix
corr = size_price_df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
print(corr)

##### Exercise 2: Find the popular app categories based on rating and no. of installs

**Hint:** [df.groupby.agg()](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html); Taking the average rating could be another approach

In [ ]:
popularAppsDF = data.groupby("Category").agg(avg_rating = ('Rating','mean'),num_of_installs = ('Installs','sum')).sort_values('avg_rating',ascending=False)
popularAppsDF.head()

##### Exercise 3: How many apps are produced in each year category-wise ?

  * Create a `Year` column by slicing the values of `Last Updated` column and find the Year with most no. of apps produced 

    **For example**, slice the year `2017` from `February 8, 2017` 

  * Find the categories which have a consistent rating in each year

      **Hint:** `sns.countplot`

In [ ]:
data['Year']=data['Last Updated'].map(lambda x: x[-4:])
appsyearwisedf = data.groupby(by=["Year","Category"]).agg(num_of_apps = ('App','nunique')).reset_index()
appsyearwisedf.pivot(index="Category",columns="Year",values="num_of_apps").fillna(0)

In [ ]:
ratingdf = data.groupby(by=["Year","Category"]).agg(avg_rating=('Rating','mean')).reset_index()
ratingdf["avg_rating"]= ratingdf["avg_rating"].map(lambda x:round(x,2))
ratingdf= ratingdf.pivot(index="Category",columns="Year",values="avg_rating").fillna(0)
ratingdf

In [ ]:
ratingdf.plot(kind="bar",figsize=(100,10),title="Ratings of categories years wise",ylabel="Average Rating")
plt.show()

##### Exercise 4: Identify the highest paid apps with a good rating

In [ ]:
data["Price_numeric"] = data['Price'].apply(lambda price : float(price.replace('$',"")))
data.sort_values(by="Price_numeric",ascending=False,inplace=True)
highestpaidapps = data.head(20)[(data.Rating>4)]
highestpaidapps

##### Exercise 5: Are the top-rated apps genuine ? How about checking reviews count of top-rated apps ?

In [ ]:
topratedapps = data[(data.Rating==5)].sort_values(by="Reviews")
topratedapps.iloc[:,:6]
# Top Rated Apps are not genuine as the review count is less than 10

##### Exercise 6: If the number of reviews of an app is very low, what could be the reason for its top-rating ?

In [ ]:
#From the above dataset, it infers despite many downloads, ratings were not given by everyone. So,if few members from the development team gives 5 then rating will be 5